In [0]:
# 02_silver_clean_transform
# Goal:
# 1) Clean and transform:
# 2) Normalize column names.
# 3) Remove invalid records (e.g., missing ticket_id).
# 4) Convert date fields to proper timestamp format.
# 5) Save as Delta table or parquet.

from pyspark.sql import functions as F

# Read from the bronze table
BRONZE_TABLE = "bronze_servicenow_incidents"
SILVER_TABLE = "silver_servicenow_incidents"

df_bronze = spark.table(BRONZE_TABLE)

print("Bronze rows:", df_bronze.count())
df_bronze.printSchema()


Bronze rows: 10000
root
 |-- number: string (nullable = true)
 |-- sys_id: string (nullable = true)
 |-- opened_at: string (nullable = true)
 |-- closed_at: string (nullable = true)
 |-- state: string (nullable = true)
 |-- priority: long (nullable = true)
 |-- impact: long (nullable = true)
 |-- urgency: long (nullable = true)
 |-- severity: string (nullable = true)
 |-- category: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- description: string (nullable = true)
 |-- assignment_group: string (nullable = true)
 |-- assigned_to: string (nullable = true)
 |-- assigned_to_sys_id: string (nullable = true)
 |-- caller: string (nullable = true)
 |-- caller_sys_id: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- location: string (nullable = true)
 |-- cmdb_ci: string (nullable = true)
 |-- sla_breached: boolean (nullable = true)
 |-- reopen_count: long (nullable = true)
 |-- u_source: string (nul

In [0]:
# Normalize column names
def normalize_columns(df):
    for col in df.columns:
        normalized = (
            col.lower()
               .replace(" ", "_")
               .replace("-", "_")
        )
        df = df.withColumnRenamed(col, normalized)
    return df

df_norm = normalize_columns(df_bronze)


In [0]:
# Remove invalid records (missing ticket id)
df_valid = (
    df_norm
    .filter(
        F.col("sys_id").isNotNull() &
        (F.trim(F.col("sys_id")) != "")
    )
)

print("After removing invalid records:", df_valid.count())


After removing invalid records: 10000


In [0]:
# Convert date fields to proper timestamp format.
df_dates = (
    df_valid
    .withColumn("opened_at_ts", F.to_timestamp("opened_at"))
    .withColumn("closed_at_ts", F.to_timestamp("closed_at"))
)


In [0]:
# Write Silver Delta table
(
    df_dates.write
      .format("delta")
      .mode("overwrite")
      .saveAsTable(SILVER_TABLE)
)

print(f"Silver Delta table created: {SILVER_TABLE}")


---------------------------------------------------------------------------
DateTimeException                         Traceback (most recent call last)
File <command-5993408257705848>, line 5
      1 (
      2     df_dates.write
      3       .format("delta")
      4       .mode("overwrite")
----> 5       .saveAsTable(SILVER_TABLE)
      6 )
      8 print(f"Silver Delta table created: {SILVER_TABLE}")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.command(self._spark.client), self._write.observations
    739 )
    740 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observati

In [0]:
# Occured in "TIMESTAMP" error. 
# The proper time format should be '2025-06-05 13:10:00' 
df_dates = (
    df_valid
    .withColumn(
        "opened_at_ts",
        F.to_timestamp("opened_at", "M/d/yyyy HH:mm")
    )
    .withColumn(
        "closed_at_ts",
        F.to_timestamp("closed_at", "M/d/yyyy HH:mm")
    )
)


In [0]:
# Write Silver Delta table again
(
    df_dates.write
      .format("delta")
      .mode("overwrite")
      .saveAsTable(SILVER_TABLE)
)

print(f"Silver Delta table created: {SILVER_TABLE}")

---------------------------------------------------------------------------
DateTimeException                         Traceback (most recent call last)
File <command-5993408257705850>, line 5
      1 (
      2     df_dates.write
      3       .format("delta")
      4       .mode("overwrite")
----> 5       .saveAsTable(SILVER_TABLE)
      6 )
      8 print(f"Silver Delta table created: {SILVER_TABLE}")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.command(self._spark.client), self._write.observations
    739 )
    740 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observati

In [0]:
# Try to convert date fields using 'try_to_timestamp'
df_dates = (
    df_valid
    .withColumn(
        "opened_at_ts",
        F.coalesce(
            F.expr("try_to_timestamp(opened_at, 'M/d/yyyy HH:mm')"),
            F.expr("try_to_timestamp(opened_at, 'M/d/yyyy H:mm')")
        )
    )
    .withColumn(
        "closed_at_ts",
        F.coalesce(
            F.expr("try_to_timestamp(closed_at, 'M/d/yyyy HH:mm')"),
            F.expr("try_to_timestamp(closed_at, 'M/d/yyyy H:mm')")
        )
    )
)


In [0]:
# Write Silver Delta table
(
    df_dates.write
      .format("delta")
      .mode("overwrite")
      .saveAsTable(SILVER_TABLE)
)

print(f"Silver Delta table created: {SILVER_TABLE}")

Silver Delta table created: silver_servicenow_incidents


In [0]:
# Confirm Silver table.
# Shows sample rows.
spark.table("silver_servicenow_incidents").printSchema()
display(spark.table("silver_servicenow_incidents").limit(20))

# Row count validation.
print("Bronze rows:", spark.table("bronze_servicenow_incidents").count())
print("Silver rows:", spark.table("silver_servicenow_incidents").count())

root
 |-- number: string (nullable = true)
 |-- sys_id: string (nullable = true)
 |-- opened_at: string (nullable = true)
 |-- closed_at: string (nullable = true)
 |-- state: string (nullable = true)
 |-- priority: long (nullable = true)
 |-- impact: long (nullable = true)
 |-- urgency: long (nullable = true)
 |-- severity: string (nullable = true)
 |-- category: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- description: string (nullable = true)
 |-- assignment_group: string (nullable = true)
 |-- assigned_to: string (nullable = true)
 |-- assigned_to_sys_id: string (nullable = true)
 |-- caller: string (nullable = true)
 |-- caller_sys_id: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- location: string (nullable = true)
 |-- cmdb_ci: string (nullable = true)
 |-- sla_breached: boolean (nullable = true)
 |-- reopen_count: long (nullable = true)
 |-- u_source: string (nullable = true)
 |-- 

number,sys_id,opened_at,closed_at,state,priority,impact,urgency,severity,category,subcategory,short_description,description,assignment_group,assigned_to,assigned_to_sys_id,caller,caller_sys_id,channel,location,cmdb_ci,sla_breached,reopen_count,u_source,tags,knowledge_linked,resolution_code,close_notes,_ingest_ts,_source_file,opened_at_ts,closed_at_ts
INC0000001,237b51cad303877ebce4b0f39d234b9a,6/5/2025 13:10,6/9/2025 11:45,Closed,3,3,3,Moderate,Software,License,License issue via Phone,User reported a license problem categorized under Software. Priority 3 with impact 3 and urgency 3. Assigned to Service Desk.,Service Desk,null,null,Morgan Smith,bdb82e00904dca6452fb67c3c889b196,Phone,Scranton,APP-ZML,false,0,User Report,"Security,Compliance,Patch",false,Workaround applied,Escalated to vendor; patch applied.,2025-12-23T16:01:48.936Z,upload_table,2025-06-05T13:10:00.000Z,2025-06-09T11:45:00.000Z
INC0000002,dc06edc0668235ba6e38facc3bbe5924,2/15/2025 3:21,null,In Progress,3,3,3,Moderate,Hardware,Desktop,Desktop issue via Email,User reported a desktop problem categorized under Hardware. Priority 3 with impact 3 and urgency 3. Assigned to Service Desk.,Service Desk,null,null,Shawn Thomas,98fbfb55f92df9200883b706adf62b78,Email,Allentown,LAPTOP-ISH926,false,0,Monitoring Alert,"VIP,ChangeRelated",true,null,null,2025-12-23T16:01:48.936Z,upload_table,2025-02-15T03:21:00.000Z,null
INC0000003,cfdfdef5207918795ef338b1e6d3791e,1/7/2025 0:22,1/9/2025 10:32,Resolved,3,3,3,Moderate,Network,WiFi,WiFi issue via Portal,User reported a wifi problem categorized under Network. Priority 3 with impact 3 and urgency 3. Assigned to Network Operations.,Network Operations,Lee Miller,357d5857b078249930a4df914da2ff9a,Pat White,cd67954226d3c53d06990b56fcc0b171,Portal,Harrisburg,LAPTOP-BNX430,false,2,Auto-Discovery,"Patch,VIP",true,Workaround applied,Escalated to vendor; patch applied.,2025-12-23T16:01:48.936Z,upload_table,2025-01-07T00:22:00.000Z,2025-01-09T10:32:00.000Z
INC0000004,d184474a7cf48dce22c8befa02eb2c6d,11/28/2024 10:20,12/2/2024 9:37,New,3,3,3,High,Access,Role Change,Role Change issue via Chat,User reported a role change problem categorized under Access. Priority 3 with impact 3 and urgency 3. Assigned to Security Operations.,Security Operations,Morgan Hernandez,0a7dade0e3e7ce73a5b4db84c41f3ec0,Jordan Johnson,1deb81c25a131bd57ea9b9fb0717483c,Chat,Harrisburg,APP-MAJ,false,0,Monitoring Alert,"External,VIP",false,User education,Escalated to vendor; patch applied.,2025-12-23T16:01:48.936Z,upload_table,2024-11-28T10:20:00.000Z,2024-12-02T09:37:00.000Z
INC0000005,51c97d2306f247e00a3d4f27c233ab94,4/10/2025 3:35,4/11/2025 23:02,Resolved,2,3,3,Critical,Software,Office Suite,Office Suite issue via Email,User reported a office suite problem categorized under Software. Priority 2 with impact 3 and urgency 3. Assigned to Service Desk.,Service Desk,null,null,Casey Thomas,cc4e5a1f3b9abd8d7a50c11f217ffec6,Email,Pittsburgh Office,LAPTOP-VRK128,false,0,Service Catalog,null,true,Configuration change,Network team resolved intermittent outage.,2025-12-23T16:01:48.936Z,upload_table,2025-04-10T03:35:00.000Z,2025-04-11T23:02:00.000Z
INC0000006,90966c917fc37f20ba4cdb5f20208611,4/21/2025 23:56,4/24/2025 6:08,In Progress,4,3,3,Low,Security,Policy,Policy issue via Phone,User reported a policy problem categorized under Security. Priority 4 with impact 3 and urgency 3. Assigned to Desktop Support.,Desktop Support,Avery Williams,6f068d9da084042adffe79567b3b295b,Hayden Jones,c99ac85fe5628a7fe4bd5ef434485528,Phone,Scranton,SRV-SPF77,false,0,User Report,null,true,User education,Workaround documented pending permanent fix.,2025-12-23T16:01:48.936Z,upload_table,2025-04-21T23:56:00.000Z,2025-04-24T06:08:00.000Z
INC0000007,d3e19530405fb85b4830ad8282feb1f5,3/29/2025 11:20,3/30/2025 3:10,New,4,3,3,Moderate,Software,Custom App,Custom App issue via Portal,User reported a custom app problem categorized under Software. Priority 4 with impact 3 and urgency 3. Assigned to Service Desk.,Service 

Bronze rows: 10000
Silver rows: 10000
